<a href="https://colab.research.google.com/github/karthikjangam/predictive_maintainence/blob/main/Copy_of_Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers wandb scikit-learn gradio flask joblib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 39.9/39.9 MB 201.3 MB/s eta 0:00:01

In [ ]:
import wandb
from transformers import TrainerCallback

# Initialize W&B
wandb.init(project="text_classification")

# Custom callback to log additional metrics to W&B
class WeightsAndBiasesCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            wandb.log(logs)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load IMDb dataset
dataset = load_dataset('imdb')

# Load pre-trained tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

train_dataset = dataset['train'].map(tokenize_function, batched=True)
test_dataset = dataset['test'].map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
# Import necessary libraries
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Specify the pre-trained model name
model_name = 'distilbert-base-uncased'  # You can replace this with 'bert-base-uncased'

# Load pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load dataset (IMDb in this case)
dataset = load_dataset('imdb')

# Reduce the dataset size (use 10% of the original data for faster training)
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(int(0.1 * len(dataset['train']))))
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(int(0.1 * len(dataset['test']))))

# Tokenize the dataset with reduced max length (e.g., 128 tokens instead of 512)
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to the smaller dataset
train_tokenized = small_train_dataset.map(tokenize_function, batched=True)
test_tokenized = small_test_dataset.map(tokenize_function, batched=True)

# Define accuracy metrics
def compute_metrics(p):
    pred, labels = p
    pred = pred.argmax(-1)
    accuracy = accuracy_score(labels, pred)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define training arguments with various optimizations
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=3,  # Limit saved checkpoints
    learning_rate=5e-5,  # Fine-tune this value to boost accuracy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reduced number of epochs to 1
    weight_decay=0.01,  # Helps prevent overfitting
    load_best_model_at_end=True,  # Automatically load the best model based on eval loss
    report_to="wandb",  # Report to W&B for logging
    fp16=True,  # Enable mixed precision training to speed up training
    gradient_accumulation_steps=4  # Perform gradient accumulation to simulate a larger batch size
)

# Trainer initialization with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement for 3 evaluations
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_tokenizer')


ModuleNotFoundError: No module named 'datasets'

In [ ]:
!pip install datasets transformers gradio scikit-learn wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
!pip install datasets transformers gradio scikit-learn wandb
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback, pipeline
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import gradio as gr

# Specify the pre-trained model name
model_name = 'distilbert-base-uncased'  # You can replace this with 'bert-base-uncased'

# Load pre-trained model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load dataset (IMDb in this case)
dataset = load_dataset('imdb')

# Reduce the dataset size (use 10% of the original data for faster training)
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(int(0.1 * len(dataset['train']))))
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(int(0.1 * len(dataset['test']))))

# Tokenize the dataset with reduced max length (e.g., 128 tokens instead of 512)
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to the smaller dataset
train_tokenized = small_train_dataset.map(tokenize_function, batched=True)
test_tokenized = small_test_dataset.map(tokenize_function, batched=True)

# Define accuracy metrics
def compute_metrics(p):
    pred, labels = p
    pred = pred.argmax(-1)
    accuracy = accuracy_score(labels, pred)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define training arguments with original optimizations
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=3,  # Limit saved checkpoints
    learning_rate=5e-5,  # Fine-tune this value to boost accuracy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,  # Reduced number of epochs to 1
    weight_decay=0.01,  # Helps prevent overfitting
    load_best_model_at_end=True,  # Automatically load the best model based on eval loss
    report_to="wandb",  # Report to W&B for logging
    fp16=True,  # Enable mixed precision training to speed up training
    gradient_accumulation_steps=4  # Perform gradient accumulation to simulate a larger batch size
)

# Trainer initialization with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement for 3 evaluations
)

# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model('fine_tuned_model')
tokenizer.save_pretrained('fine_tuned_tokenizer')

# Experiment with hyperparameter tuning: different learning rate, batch size, and epochs
tuning_args = TrainingArguments(
    learning_rate=3e-5,  # Experiment with a lower learning rate
    num_train_epochs=5,  # Train for more epochs
    per_device_train_batch_size=32,  # Increase batch size
    output_dir='./tuned_results',
)

# Re-initialize trainer with new hyperparameters
tuned_trainer = Trainer(
    model=model,
    args=tuning_args,
    train_dataset=train_tokenized,  # Use tokenized datasets
    eval_dataset=test_tokenized,    # Use tokenized datasets
    compute_metrics=compute_metrics,
)

# Train again with new hyperparameters
tuned_trainer.train()

# Save the newly tuned model
tuned_trainer.save_model('tuned_fine_tuned_model')

# Load the fine-tuned model and tokenizer for inference
classifier = pipeline('text-classification', model='tuned_fine_tuned_model', tokenizer='fine_tuned_tokenizer')

# Make a prediction
text = "The movie was fantastic! I loved every moment of it."
prediction = classifier(text)
print(f"Prediction: {prediction[0]['label']} with confidence {prediction[0]['score']:.4f}")

# Define a function to classify text using Gradio
def classify_text(text):
    result = classifier(text)
    # Map LABEL_0 to "negative" and LABEL_1 to "positive"
    label_mapping = {'LABEL_0': 'negative', 'LABEL_1': 'positive'}
    return label_mapping[result[0]['label']]  # Return mapped label (human-readable)

# Define Gradio interface (updated to use gr.components)
iface = gr.Interface(
    fn=classify_text,
    inputs=gr.Textbox(lines=2, placeholder='Enter text here...'),
    outputs=gr.Textbox(),
    title="Text Classification Model",
    description="Enter text to get predictions on sentiment."
)

# Launch the Gradio app
iface.launch()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,No log,0.427066,0.813200,0.761936,0.914081,0.831103


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prediction: LABEL_1 with confidence 0.9943
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://304b3ff8228e161096.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
